# Access Estimate

TODO:
- Get proper NTL from website (and mosaic)
- Loop through combined.gpkg

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio

from gridfinder._util import save_raster
from accessestimator.access_rates import regularise, estimate

## Setup

In [ ]:
country = 'COL'
access = {'total': 0.99, 'urban': 1.00, 'rural': 0.96}

In [ ]:
targets_in = 'targets.tif'
admin_in = 'ne_50m_admin0_buf.gpkg'
pop_in = 'ghs.tif'
urban_in = 'urb.tif'
ntl_in = 'ntl.tif'

## Regularise

In [ ]:
admin = gpd.read_file(aoi_in)
aoi = admin.loc[admin["ADM0_A3"] == country]
aoi.plot()

In [ ]:
pop, urban, ntl, targets, affine, crs = regularise(country, aoi, pop_in, urban_in, ntl_in, targets_in)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,12))
axes =     [ax1,   ax2,     ax3,   ax4]
rasters =  [pop,   urban,   ntl,   targets]
titles =   ['pop', 'urban', 'ntl', 'targets']
maxes =    [1000,  None,    1,     None]

for ax, raster, title, vmax in zip(axes, rasters, titles, maxes):
    ax.imshow(raster, vmax=vmax)
    ax.set_title(title)
plt.show()

## Run the algorithm

In [ ]:
pop_elec, access_model_total, weights = estimate(pop, urban, ntl, targets, access)

In [ ]:
save_raster(f'pop_elec_{country}.tif', pop_elec, affine, crs)
save_raster(f'weights_{country}.tif', weights, affine, crs)